In [1]:
# will autoupdate any of the packages imported:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
import pyclifford as pc

In [3]:
from numba import njit

In [57]:
import seaborn as sns
sns.set_theme()

In [112]:
native_bond_length = (0.11779+0.47116)**2+0.75545**2
ratio = 1.5/native_bond_length
molecule = [["O", [0.00000, 0.00000, 0.11779*ratio]], \
            ["H", [0.00000, 0.75545*ratio, -0.47116*ratio]],\
            ["H", [0.00000, -0.75545*ratio, -0.47116*ratio]]]
# bond_len = 2.0
# molecule = [["Cr", [0, 0.0, 0]], ["Cr", [bond_len, 0, 0]]]
pyc_h, E_exact,shift = pc.qchem_hamiltonian(molecule,use_pyscf=True)

Done calculating qubit_op.
Done calculating pyclifford Hamiltonian.
Using pyscf ground state estimate
converged SCF energy = -74.6602398552415


In [113]:
pyc_h.gs.shape

(550, 20)

In [114]:
@njit 
def compatible(g1, g2):
    if g1.shape[0]!=g2.shape[0]:
        raise ValueError("Two Pauli strings have different number of qubits.")
    N = g1.shape[0]//2
    compatible = True
    for ii in range(N):
        if (g1[ii*2],g1[ii*2+1])!=(0,0):
            if (g2[ii*2],g2[ii*2+1])!=(0,0):
                if (g1[ii*2],g1[ii*2+1])!=(g2[ii*2],g2[ii*2+1]):
                    compatible = False
    return compatible
@njit
def union_pauli(g1,g2):
    if compatible(g1,g2)==False:
        raise ValueError("Non-compatible Pauli operators does not have union.")
    unions = np.zeros_like(g1)
    N = g1.shape[0]//2
    for ii in range(N):
        if (g1[ii*2],g1[ii*2+1])==(g2[ii*2],g2[ii*2+1]):
            unions[2*ii]=g1[2*ii]
            unions[2*ii+1]=g1[2*ii+1]
        else:
            unions[2*ii]=g1[2*ii]+g2[2*ii]
            unions[2*ii+1] = g1[2*ii+1]+g2[2*ii+1]
    return unions

In [115]:
compatible(pc.pauli("ZIX").g, pc.pauli("ZYX").g)

True

In [116]:
union_pauli(pc.pauli("ZIX").g, pc.pauli("ZYX").g)

array([0, 1, 1, 1, 1, 0])

In [117]:
@njit
def Overlap_Set(abs_H_cs, H_gs):
    sort_idx = np.argsort(abs_H_cs)[::-1]
    abs_alpha = abs_H_cs[sort_idx]
    gs = H_gs[sort_idx]
    terms = abs_H_cs.shape[0]
    occupation = np.zeros(terms) # denote whether a pauli is in the set or not
    Es = [] # set of obervables
    Ps = [] # set of measurement basis
    Ws = [] # initial weights
    j = 0
    s = 0
    while j<terms: #np.sum(occupation)!=terms:
        tmp_Ps=gs[j]
        tmp_es = []
        tmp_ws = 0
        tmp_es.append(gs[j])
        occupation[j]=1
        tmp_ws += abs_alpha[j]
        s += 1
        for k in range(j+1,terms):
            if compatible(gs[k],tmp_Ps): # if observable Q(k) is compatible with P(s)
                tmp_es.append(gs[k])
                occupation[k]=1
                tmp_ws += abs_alpha[k]
                tmp_Ps = union_pauli(tmp_Ps,gs[k])
        for k in range(j):
            if compatible(gs[k],tmp_Ps):
                tmp_es.append(gs[k])
                occupation[k]=1
                tmp_ws += abs_alpha[k]
                tmp_Ps = union_pauli(tmp_Ps,gs[k])
        Ps.append(tmp_Ps)
        Es.append(tmp_es)
        Ws.append(tmp_ws)
        # update j
        still_leftover = False
        for p in range(j+1,terms):
            if occupation[p]==0:
                j = p
                still_leftover = True
                break
        if not still_leftover:
            j = terms
                
    return Es, Ps, Ws

In [118]:
def translate_overlap_set(Es, Ps):
    p_gs = []
    for i in range(len(Ps)):
        p_gs.append(Ps[i].tolist())
    new_Ps = pc.paulialg.PauliList(gs = np.array(p_gs))
    new_Es = []
    for i in range(len(Es)):
        tmp = []
        for j in range(len(Es[i])):
            tmp.append(Es[i][j].tolist())
        new_Es.append(pc.paulialg.PauliList(gs=np.array(tmp)))
    return new_Es, new_Ps

In [119]:
abs(pyc_h.cs).shape

(550,)

In [120]:
# H = 0.25*pc.pauli("XXI")+0.25*pc.pauli("IXX")+(1./12.)*pc.pauli("IXZ")+(1./4.)*pc.pauli("IZZ")\
# +(1./12.)*pc.pauli("ZIZ")+(1./12.)*pc.pauli("XIZ")

In [121]:
Es, Ps, Ws = Overlap_Set(abs(pyc_h.cs),pyc_h.gs)

In [122]:
Es, Ps = translate_overlap_set(Es,Ps)

In [124]:
len(Ps)

174

In [109]:
total_terms = 0
for i in range(len(Es)):
    total_terms += Es[i].gs.shape[0]

In [110]:
total_terms

1752379

In [84]:
pyc_h.gs.shape[0]

550

In [132]:
def test(a, *qubits):
    print(a)
    print(*qubits)

In [133]:
test(1,2,3,4)

1
2 3 4
